In [ ]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.research.experimental import continuous_future, history

In [ ]:
def get_hist(symbol, start, end, num_of_maturities):
        df = history(
            [continuous_future(symbol,
                               offset=i,
                               adjustment=None) for i in range(num_of_maturities)],
            fields='price', 
            frequency='daily', 
            start=start, 
            end=end
        )
        return df

def get_remain_days(symbols, start, end,):
    df = history( [continuous_future(symbol, offset=0) for symbol in symbols],
                 fields='contract',
                  frequency='daily', 
                 start=start,
                 end=end)
    return df
    
    

In [ ]:
symbols = ['CL', 'HO', 'XB', 'NG']
symbols_pairs = list(itertools.combinations(symbols, 2))
start = '2015-01-01'
end = '2018-05-31'


In [ ]:
hist_df = [get_hist(symbol,
                    start, end, 5) for symbol in symbols]

In [ ]:
maturities_df = [pd.concat(
    [df.iloc[:, i] for df in hist_df], axis=1)
                 for i, x in enumerate(hist_df[0].columns)]

In [ ]:
for df in maturities_df:
    df.columns = symbols

In [ ]:
df_contracts = get_remain_days(symbols, start, end)
df_contracts.columns = symbols
df_expiration_date =  df_contracts.applymap(lambda x: x.expiration_date)

In [ ]:
# XB と HO の残存期間がなぜかあわない日
# suspicious = (df_expiration_date["XB"] - df_expiration_date["HO"]).astype(int) != 0
#df_expiration_date[suspicious]

df_expiration_date["today"]  = df_expiration_date.index
df_remain_days = pd.DataFrame({'CL':(df_expiration_date["CL"] - df_expiration_date["today"]).apply(lambda x:x.days),
              'XB':(df_expiration_date["XB"] - df_expiration_date["today"]).apply(lambda x:x.days),
              'HO':(df_expiration_date["HO"] - df_expiration_date["today"]).apply(lambda x:x.days),
              'NG':(df_expiration_date["NG"] - df_expiration_date["today"]).apply(lambda x:x.days),})



In [ ]:
def plot_ratio(maturity):
    fig = plt.figure(figsize=(14, 14))
    fig.suptitle('offset:{}'.format(maturity))
    ax = [fig.add_subplot(len(symbols_pairs) + 1, 1, i) for i in range(1, len(symbols_pairs) + 1)]
    for i, symbol in enumerate(symbols_pairs):
        ax[i].plot(maturities_df[maturity][list(symbol)].apply(lambda x: x[0] / x[1], axis=1))
        ax[i].set_title('{}/{}'.format(*symbol))

In [ ]:
def plot_ratio_by_symbols(maturities_df, symbol1, symbol2):
    fig = plt.figure()
    fig.suptitle('symbols:{}/{}'.format(symbol1, symbol2))
    ax = [fig.add_subplot(len(maturities_df) + 1, 1, i) for i in range(1,len(maturities_df)+ 1)]
    for i in range(5):
        ax[i].plot(maturities_df[i][symbol1] / maturities_df[i][symbol2], label=i)
        ax[i].legend()

def plot_ratio_by_symbols_with_remain_date(maturities_df, symbol1, symbol2, df_remain_days):
    fig = plt.figure()
    fig.suptitle('symbols:{}/{}'.format(symbol1, symbol2))
    ax = [fig.add_subplot(len(maturities_df) + 1, 1, i) for i in range(1,len(maturities_df)+ 1)]
    for i in range(len(maturities_df)):
        ax[i].plot(maturities_df[i][symbol1] / maturities_df[i][symbol2], label=i)
        ax2 = ax[i].twinx()
        ax2.plot(df_remain_days[[symbol1, symbol2]])
        ax[i].legend()
        

In [ ]:
plot_ratio_by_symbols_with_remain_date(maturities_df, "HO", "XB", df_remain_days)
plot_ratio_by_symbols(maturities_df, "HO", "XB")


In [ ]:
plot_ratio(1)

In [ ]:
plot_ratio(2)

In [ ]:
plot_ratio(3)

In [ ]:
plot_ratio(4)

In [ ]:
ho_xb_5 = maturities_df[4][['HO', 'XB']]
ho_xb_5_ratio = ho_xb_5['HO'] / ho_xb_5['XB']
ho_xb_5_ratio.describe()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2, sharey=ax1)
ax1.plot(ho_xb_5)
ax2.plot(ho_xb_5_ratio)

In [ ]:
ho_xb_5_ratio.hist(bins=100)

In [ ]:
returns = ho_xb_5_ratio.pct_change(10).shift(-10)
short_returns = ((returns[ho_xb_5_ratio > 1 + ho_xb_5_ratio.std()]) * -1).cumsum()
short_returns.plot()

In [ ]:
returns[ho_xb_5_ratio > (1 + ho_xb_5_ratio.std())]
1 + ho_xb_5_ratio.std()